# Compare USDM & SPI
a. stein 4.18.2022

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import xarray as xr
import rioxarray
import rasterio as rio
import pandas as pd
import geopandas as gpd

from tqdm.autonotebook import tqdm

import sys
sys.path.append('../../')
import sedi.wrangle as wrangle

Populating the interactive namespace from numpy and matplotlib


/pool0/data/steinadi/.conda/envs/sedi/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if sys.path[0] == '':


Load in USDM and SPI

In [2]:
dm_path = '/pool0/home/steinadi/data/drought/drought_impact/data/drought_measures'

usdm = xr.open_dataset(f'{dm_path}/usdm/USDM_20000104_20220418.nc')
spi_1y = xr.open_dataset(f'{dm_path}/spi/spi1y.nc')

In [3]:
usdm_da = usdm['USDM'].rio.write_crs('EPSG:4326', inplace=True)
spi_da = spi_1y['spi'].rio.write_crs('EPSG:4326', inplace=True)

Grab some bounds for Washington and check the crs

In [4]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
states_gdf = gpd.read_file(states_url)
wa_gdf = states_gdf[(states_gdf.NAME=='Washington')]

In [5]:
wa_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Clip USDM and SPI to WA

In [6]:
spi_da_wa = wrangle.clip_xarray(spi_da, wa_gdf)

In [7]:
usdm_da

<xarray.DataArray 'USDM' (date: 1163, lat: 8568, lon: 14358)>
[143071497072 values with dtype=float64]
Coordinates:
  * lat          (lat) float64 17.87 17.87 17.87 17.87 ... 71.4 71.4 71.4 71.4
  * date         (date) datetime64[ns] 2000-01-04 2000-01-11 ... 2022-04-12
  * lon          (lon) float64 -179.1 -179.1 -179.1 -179.1 ... 179.7 179.8 179.8
    spatial_ref  int64 0
Attributes:
    long_name:     United States Drought Monitor
    catagories:    -1:Neutral (or Wet), 0:D0 (Abnormal Dry), 1:D1 (Moderate D...
    grid_mapping:  spatial_ref

In [8]:
#usdm_da = usdm_da.rename({'x':'lon', 'y':'lat'})

ValueError: cannot rename 'x' because it is not a variable or dimension in this dataset

In [15]:
wrangle.clip_xarray(usdm_da, wa_gdf)

MissingSpatialDimensionError: y dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'y' can address this. Data variable: USDM

Well that's unfortunate

In [18]:
wrangle.clip_xarray(usdm_da.rio.set_spatial_dims('lon','lat'), wa_gdf)

MissingSpatialDimensionError: y dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'y' can address this. Data variable: USDM

Hmmm ... don't like that

In [19]:
usdm_da_renamed = usdm_da.rename({'lon':'x', 'lat':'y'})
wrangle.clip_xarray(usdm_da_renamed, wa_gdf)

MemoryError: Unable to allocate 533. GiB for an array with shape (1163, 8568, 14358) and data type int32

It's not an `int32`? Also yikes on that size ... I'm going to check how big the netcdf is ...

In [14]:
usdm_da.nbytes/1E9

1144.571976576

ITS A TERABYTE :O

In [20]:
usdm_da_renamed

<xarray.DataArray 'USDM' (date: 1163, y: 8568, x: 14358)>
[143071497072 values with dtype=float64]
Coordinates:
  * y            (y) float64 17.87 17.87 17.87 17.87 ... 71.4 71.4 71.4 71.4
  * date         (date) datetime64[ns] 2000-01-04 2000-01-11 ... 2022-04-12
  * x            (x) float64 -179.1 -179.1 -179.1 -179.1 ... 179.7 179.8 179.8
    spatial_ref  int64 0
Attributes:
    long_name:     United States Drought Monitor
    catagories:    -1:Neutral (or Wet), 0:D0 (Abnormal Dry), 1:D1 (Moderate D...
    grid_mapping:  spatial_ref

What about just one time?

In [22]:
wrangle.clip_xarray(usdm_da_renamed.isel(date=0), wa_gdf)

<xarray.DataArray 'USDM' (y: 554, x: 314)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * y            (y) float64 45.55 45.55 45.55 45.57 ... 48.97 49.0 49.0 49.0
  * x            (x) float64 -124.7 -124.7 -124.7 ... -117.0 -116.9 -116.9
    date         datetime64[ns] 2000-01-04
    spatial_ref  int64 0
Attributes:
    long_name:     United States Drought Monitor
    catagories:    -1:Neutral (or Wet), 0:D0 (Abnormal Dry), 1:D1 (Moderate D...
    grid_mapping:  spatial_ref

Hmmm ... that's fine ... how big can we get?

In [23]:
wrangle.clip_xarray(usdm_da_renamed.isel(date=np.arange(0,500)), wa_gdf)

MemoryError: Unable to allocate 458. GiB for an array with shape (500, 8568, 14358) and data type float64

So it seems to have an issue that it's then taking up a lot of memory ... maybe I need to scatter the processing for big files then?

4.19.2022 - coming back to this ... but skipping all the errors above.

when I think about this, scattering won't solve the issue of it taking up too much memory ... there needs to be improved loading & unloading, but scattering could help speed up the process beyond a linear train.

I'm going to jump into a new notebook to create some clipped versions of the USDM and see if I can reduce the file size